In [35]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image
from datasets import load_dataset, load_from_disk
import os
from pathlib import Path

dataset = load_from_disk("/home/ubuntu/LLaVA-Graph/with_abstract_graph_derived_question_first_one_percent_train")
img_root = "/home/ubuntu/imgs/train/"

image_files = [Path(img_root + item) for item in dataset['image_file']]
print(f'image_files: {image_files}')

image_files: [PosixPath('/home/ubuntu/imgs/train/1801.01974v2-Figure10-1.png'), PosixPath('/home/ubuntu/imgs/train/1801.05671v1-Figure9-1.png'), PosixPath('/home/ubuntu/imgs/train/2004.06653v1-Figure9-1.png'), PosixPath('/home/ubuntu/imgs/train/1803.06598v1-Figure8-1.png'), PosixPath('/home/ubuntu/imgs/train/2004.04071v2-Figure7-1.png'), PosixPath('/home/ubuntu/imgs/train/1603.08982v1-Figure2-1.png'), PosixPath('/home/ubuntu/imgs/train/1411.4940v2-Figure9-1.png'), PosixPath('/home/ubuntu/imgs/train/2005.11194v2-Figure4-1.png'), PosixPath('/home/ubuntu/imgs/train/1103.5426v3-Figure7-1.png'), PosixPath('/home/ubuntu/imgs/train/1301.0216v1-Figure9-1.png'), PosixPath('/home/ubuntu/imgs/train/1610.06815v1-Figure4-1.png'), PosixPath('/home/ubuntu/imgs/train/1904.07061v1-Figure13-1.png'), PosixPath('/home/ubuntu/imgs/train/1904.05777v2-Figure6-1.png'), PosixPath('/home/ubuntu/imgs/train/1804.02167v1-Figure3-1.png'), PosixPath('/home/ubuntu/imgs/train/1910.11851v1-Figure6-1.png'), PosixPath('/

In [37]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, transforms=None):
        self.image_paths = image_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        
        if self.transforms:
            image = self.transforms(image)
            
        return image

In [19]:
os.path.exists(image_files[0])

False

In [38]:

import torch
import clip
from PIL import Image
import activeloop 


device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-L/14", device=device)


# transforms = transforms.Compose([
#     transforms.Resize(224),
#     transforms.ToTensor()
# ])

dataset = CustomImageDataset(image_files, preprocess)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)
with torch.no_grad():
    for imgs in dataloader:
        imgs = imgs.to(device)
        embeddings = model.encode_image(imgs)
        break



In [42]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore


In [50]:
embeddings.shape

torch.Size([64, 768])

In [58]:
from PIL import Image


# 
# imgs.detach().cpu().numpy()[0].shape

Image.fromarray(imgs.detach().cpu().numpy()[0])

TypeError: Cannot handle this data type: (1, 1, 224), <f4

In [53]:
import deeplake
import numpy as np
from tqdm import tqdm
dataset_path = "hub://findalexli/test_clip_embedding"


with deeplake.empty(dataset_path, overwrite=True) as ds:
    # Create the tensors 
    ds.create_tensor('images', htype = 'image', 
                     sample_compression = 'jpeg')
    ds.create_tensor('embeddings')

    # Add arbitrary metadata - Optional
    ds.info.update(description = 'Dog breeds embeddings dataset')
    ds.images.info.update(camera_type = 'SLR')


    detached_imgs = imgs.detach().cpu().numpy()
    detached_embeddings = embeddings.detach().cpu().numpy()
                     
    # Iterate through the images and their corresponding embeddings,
    # and append them to hub dataset    
    for i in tqdm(range(len(detached_imgs))):

      img = detached_imgs[i]
      img = img.astype(np.uint8)

      # Append to Deep Lake Dataset
      ds.images.append(img)  
      ds.embeddings.append(detached_embeddings[i])

Your Deep Lake dataset has been successfully created!


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/findalexli/test_clip_embedding


hub://findalexli/test_clip_embedding loaded successfully.


  0%|          | 0/64 [00:00<?, ?it/s]


SampleAppendError: Failed to append a sample to the tensor 'images'. See more details in the traceback.